In [27]:
#import needed libraries
import numpy as np
import math
import collections

In [28]:
def ShannonEntropy(inputString):
  length = len(inputString)
  bases = collections.Counter([tmp_base for tmp_base in inputString])
  entropy = 0.0
  for base in bases:
    # number of residues
    n_i = bases[base]
    # n_i (# residues type i) / M (# residues in column)
    p_i = n_i / float(length)
    entropy_i = p_i * (math.log(p_i, 2))
    entropy += entropy_i
  if entropy == 0:
    return entropy
  return entropy * -1

In [29]:
#Installing libraries for ranking the text and getting the most important words
!pip install pytextrank
import pytextrank
import spacy
from icecream import ic

In [30]:
#Testing the English analyzer
engnlp = spacy.load("en_core_web_sm")
engnlp.add_pipe("textrank")

text = "Let’s say we have a language model that has been trained with a vocabulary of only 5 words sunny, day, beautiful, scenery, clouds. Now, we want to calculate the perplexity of the model when it sees the phrase beautiful scenery."
processedText = engnlp(text)

sum = 0
for phrase in processedText._.phrases:
    ic(phrase.rank, phrase.count, phrase.text)
    sum = sum + phrase.rank

#Phrase rank Average. The higher Average means the text is less likely to be artificial
average = sum / len(processedText._.phrases)
print(average)

ic| phrase.rank: 0.13708962097300056
    phrase.count: 1
    phrase.text: 'the phrase beautiful scenery'
ic| phrase.rank: 0.11032088400982727
    phrase.count: 1
    phrase.text: 'day, beautiful, scenery, clouds'
ic| phrase.rank: 0.07738845091750776
    phrase.count: 1
    phrase.text: 'a language model'
ic| phrase.rank: 0.05657660903281399
    phrase.count: 1
    phrase.text: 'the model'
ic| phrase.rank: 0.05054979272207172
    phrase.count: 1
    phrase.text: 'a vocabulary'
ic| phrase.rank: 0.03857532531560477
    phrase.count: 1
    phrase.text: 'the perplexity'
ic| phrase.rank: 0.037999529333363036
    phrase.count: 1
    phrase.text: 'only 5 words'
ic| phrase.rank: 0.0, phrase.count: 1, phrase.text: 'it'
ic| phrase.rank: 0.0, phrase.count: 1, phrase.text: 'that'
ic| phrase.rank: 0.0, phrase.count: 2, phrase.text: 'we'
ic| phrase.rank: 0.0, phrase.count: 1, phrase.text: '’s'


0.04622729202765355


In [31]:
import statistics

list = []
for phrase in processedText._.phrases:
    list.append(phrase.rank)

#The lower Variance means the text is less likely to be edited
print(len(list))
print(statistics.variance(list))

11
0.0022134860338990803


In [32]:
#Testing the Ukrainian analyzer
#!spacy download uk_core_news_sm
#ukrnlp = spacy.load("uk_core_news_sm")
#ukrnlp.add_pipe("textrank")
engnlp = spacy.load("en_core_web_sm")
engnlp.add_pipe("textrank")

text = "Маємо речення, яке ми хочемо проаналізувати. Нехай в нас є модель, натренована українською мовою. Нам необхідно отримати список рангів"
processedText = engnlp(text)
#The Ukrainian analizer didn't work, but the English one can work with sentences in Ukrainian
sum = 0
for phrase in processedText._.phrases:
    ic(phrase.rank, phrase.count, phrase.text)
    sum = sum + phrase.rank

#Phrase rank Average. The higher Average means the text is less likely to be artificial
average = sum / len(processedText._.phrases)
print(average)

ic| phrase.rank: 0.22551339760351258
    phrase.count: 1
    phrase.text: 'натренована українською мовою'
ic| phrase.rank: 0.21552814840638496
    phrase.count: 1
    phrase.text: 'українською мовою'
ic| phrase.rank: 0.19288925400862628
    phrase.count: 1
    phrase.text: 'хочемо проаналізувати'
ic| phrase.rank: 0.18006512794501117
    phrase.count: 1
    phrase.text: 'Нам необхідно отримати список рангів'
ic| phrase.rank: 0.17930495508687905
    phrase.count: 1
    phrase.text: 'список рангів'
ic| phrase.rank: 0.16303438810847415
    phrase.count: 1
    phrase.text: 'Нехай в нас є модель'
ic| phrase.rank: 0.12412561754091966
    phrase.count: 1
    phrase.text: 'хочемо'
ic| phrase.rank: 0.10705477344677523
    phrase.count: 1
    phrase.text: 'Маємо речення'
ic| phrase.rank: 0.09643570488844748
    phrase.count: 1
    phrase.text: 'яке'


0.16488348522611448


In [33]:
#Predicting the next words. Installing libraries
#https://stackoverflow.com/questions/76397904/generate-the-probabilities-of-all-the-next-possible-word-for-a-given-text

!pip install transformers

import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

In [34]:
delimiters = [',', '|', ';', '.', ':', ' ']
signs = ['\'', '\"', ' ']

#Remove last symbol from the string if it's a delimiter. Made for more clear word comparing
def ClearElements(elements):
  array = []
  for element in elements:
    if element[-1] in delimiters:
        element = element[:-1]
    array.append(element)
  return array

#Gets the array of words. Returns the string made out of these words combined
def CombineWordsInString(words):
  string = ""
  for word in words:
    if isinstance(word, int):  #If word is a number, it is converted into a string
      word = str(word)
    string = string + word + " "
  string = string[:-1]
  return string

#Clear the string from the symbols put in signs array
def ClearString(inputString):
  for item in signs:
    inputString = inputString.replace(item, '')
  return inputString

def CombineCharactersToString(inputSymbols):
  string = ""
  for character in range(1, len(inputSymbols), 2):
    string = string + inputSymbols[character]
  return string

In [35]:
#Encoding and decoding must be reworked as well - it's a different scheme
#BUT I won't encode the prompt for now. I'll encode just the sentences

def CombineSentencesToString(inputSentences):
  outputString = ""
  for sentence in inputSentences:
    outputString = outputString + " " + sentence
  return outputString

In [36]:
#Find the words possible to appear after the string we give
def FindWordsDictionary(encoded_text, numOfWords, model, tokenizer):
  #1. step to get the logits of the next token
  with torch.inference_mode():
    outputs = model(**encoded_text)
  next_token_logits = outputs.logits[0, -1, :]
  # 2. step to convert the logits to probabilities
  next_token_probs = torch.softmax(next_token_logits, -1)
  # 3. step to get the top words - numOfWords is the amount
  topk_next_tokens= torch.topk(next_token_probs, numOfWords)
  #putting it together
  dictionary = {tokenizer.decode(idx): prob for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)}
  #We also need to clear the words from the symbols - for the proper comparing
  clearedKeys = []
  for item in dictionary.keys():
    clearedKeys.append(ClearString(item))
  finDictionary = dict(zip(clearedKeys, dictionary.values()))
  return finDictionary

In [37]:
import re

def CodeAppending(words, wordsWithProbs, currentIndex, maxIndex, code):
  wordCode = -1
  #Is the next word from the sentence present in the dictionary
  isWordPresentInVector = words[currentIndex] in wordsWithProbs.keys()
  if isWordPresentInVector:
    #Find the index of the element - it will become the code
    for i, key in enumerate(wordsWithProbs.keys()):
      if key == words[currentIndex]:
        wordCode = i
        break
  #The words-numbers like 20 or -14.5 get put in the specific brackets and aquire the letter on the front
  #The decoder will see this letter and know it's a word-number, not a word code
  if words[currentIndex].isdigit():  #for integers
    code.append(numberWordKeyS)
    code.append("n" + words[currentIndex])
    code.append(numberWordKeyE)
  elif re.match(r"[-+]?(?:\d*\.*\d+)", words[currentIndex]):  #for floating-point numbers
    code.append(numberWordKeyS)
    code.append("n" + words[currentIndex])
    code.append(numberWordKeyE)
  #If the code is less than the maximum allowed, put it in code
  elif isWordPresentInVector and wordCode <= maxIndex and wordCode >= 0:
    code.append(wordCode)
  #If the word is present in the dictionary, but the code is greater than the maximum allowed
  #The word is put into the code between the specific brackets
  elif isWordPresentInVector and wordCode > maxIndex:
    code.append(notFitKeyS)
    code.append(words[currentIndex])
    code.append(notFitKeyE)
  #If the word didn't appear in the dictionary, it gets put in code between the corresponding brackets
  else:
    code.append(unknownKeyS)
    code.append(words[currentIndex])
    code.append(unknownKeyE)

In [38]:
#GPT2Tokenizer has a model trained for Ukrainian language. Link: https://huggingface.co/benjamin/gpt2-wechsel-ukrainian/blame/main/tokenizer_config.json
tUkr = GPT2TokenizerFast.from_pretrained("benjamin/gpt2-wechsel-ukrainian")
mUkr = GPT2LMHeadModel.from_pretrained("benjamin/gpt2-wechsel-ukrainian")
#Test string
encoded_text = tUkr("Мені все вдається легко, тому", return_tensors="pt")
#The scheme is the same
#1. step to get the logits of the next token
with torch.inference_mode():
  outputsUkr = mUkr(**encoded_text)
next_token_logits = outputsUkr.logits[0, -1, :]
print(next_token_logits.shape)
# 2. step to convert the logits to probabilities
next_token_probs = torch.softmax(next_token_logits, -1)
# 3. step to get the top 20
topk_next_tokens = torch.topk(next_token_probs, 20)
#putting it together
print(*[(tUkr.decode(idx), prob) for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)], sep="\n")

torch.Size([50257])
(' що', tensor(0.4398))
(' я', tensor(0.0823))
(',', tensor(0.0243))
(' й', tensor(0.0208))
(' не', tensor(0.0199))
(' і', tensor(0.0193))
(' ми', tensor(0.0110))
(' рекомендую', tensor(0.0098))
(' в', tensor(0.0086))
(' раджу', tensor(0.0075))
(' мені', tensor(0.0072))
(' з', tensor(0.0065))
(' ви', tensor(0.0046))
(' це', tensor(0.0046))
(' якщо', tensor(0.0045))
(' у', tensor(0.0042))
(' буду', tensor(0.0039))
(' все', tensor(0.0037))
(' як', tensor(0.0036))
(' дякую', tensor(0.0036))


In [ ]:
#GENERATE the text using the GPT-2 model
#Link to code: https://medium.com/@majd.farah08/generating-text-with-gpt2-in-under-10-lines-of-code-5725a38ea685

modelForGeneration = GPT2LMHeadModel.from_pretrained("benjamin/gpt2-wechsel-ukrainian", pad_token_id=tUkr.eos_token_id)
#The prompt for generation
prompt = "Навесні всі каштани вздовж нашої вулиці вкривалися білими та рожевими квітками"
#Encode the prompt
encoded_text = tUkr.encode(prompt, return_tensors="pt")
#Put it into model to generate the text
output = modelForGeneration.generate(encoded_text, max_length=300, do_sample=True, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
finalText = tUkr.decode(output[0], skip_special_tokens=True)
#Print the text
print(finalText)

Навесні всі каштани вздовж нашої вулиці вкривалися білими та рожевими квітками. Це було свято весни.
На початку весни, коли природа прокидається від зимової сплячки, ми святкуємо День весняного рівнодення. Саме в цей день ми відзначаємо свято всіх закоханих, а також тих, хто любить і поважає один одного. Цей день є символом любові, вірності, ніжності та вірності. З давніх-давен наші предки вважали, що саме в цю пору року весна пробуджує природу від зимового сну і дарує надію на краще майбутнє. Наші пращури вірили, якщо на Вербну неділю не буде ясного дня, то зима буде холодною і сніжною. Тому, щоб цього дня не було холодно, в народі говорили: «Верба на вербі, верба в хаті».
У народі існує повір'я: якщо в перший день весни погода була ясною і сонячною – весна буде довгою і щедрою на врожай. Якщо ж небо було похмурим і вкрите хмарами – літо буде дощовим і вітряним. А якщо день був ясним та безхмарним – осінь буде теплою і сухою. У народі кажуть: “Яка погода, така й зима”.
В Україні з дав

In [39]:
#Mount Google Drive and input the .tsv file with the texts and their prompts
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
dataframe = pd.read_csv('/content/gdrive/My Drive/Kaggle/GPT-2_Data_From_Text_Generation.tsv', sep='\t', header=None)
dataframe.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,0,1
0,"Сонячний літній ліс, в якому я люблю гуляти","Сонячний літній ліс, в якому я люблю гуляти, -..."
1,"Сонячний літній ліс, в якому я люблю гуляти","Сонячний літній ліс, в якому я люблю гуляти, с..."
2,Сьогодні почалася війна. Я прокинулася від зву...,Сьогодні почалася війна. Я прокинулася від зву...
3,"Сонячний літній ліс, в якому я люблю гуляти","Сонячний літній ліс, в якому я люблю гуляти. Т..."
4,"Сонячний літній ліс, в якому я люблю гуляти","Сонячний літній ліс, в якому я люблю гуляти, –..."


In [40]:
#Get the, for example, last element of the dataframe
#This element will be examined in detail
generatedText = dataframe[1].iloc[-1]
print(generatedText)
#All the other elements will be examined in the end in a loop of measure-encode-compress-measure-decode

Сонячний літній ліс, в якому я люблю гуляти, – це місце, куди хочеться повертатися знову і знову. Я ніколи не забуду цього чарівного місця, де я народилася і виросла. Тут я знайшла себе, тут я зустріла свою другу половинку. Я завжди мріяла про те, щоб у мене була своя сім’я. Але коли я стала дорослою, я зрозуміла, що це не так просто. І мені захотілося зробити щось більше, ніж просто бути мамою. У мене з’явилося багато друзів, з якими я можу поділитися своїми захопленнями, своїми думками і переживаннями. Мені дуже подобається подорожувати, і я дуже рада, коли мої друзі приїжджають до мене в гості. Коли я приїжджаю до них, мені хочеться поділитися з ними своєю любов’ю і турботою про мене. Тому я завжди буду рада знайомству з новими людьми, які захочуть поділитися своїм теплом і ніжністю з моїми близькими людьми. Це дуже важливо для мене, тому що я відчуваю, як мої близькі люди відчувають мою любов і турботу. А це дуже приємно для нас обох. Адже ми любимо один одного і дуже хочемо бути р

In [41]:
promptSizeUkr = 15  #The maximum length of a prompt
maxIndexUkr = 248  #Max code to insert in the code list
unknownKeyS = 254
unknownKeyE = 255  #If the word doesn't appear in the tensor - we put it in the code, bracketing it with these two keys
notFitKeyS = 252
notFitKeyE = 253  #If the word appeared in tensor, but didn't fit within the code limit - we put it in code, bracketing with these two keys
numberWordKeyS = 250
numberWordKeyE = 251  #If the word is a number and doesn't have to be decrypted while decoding, we put it in code, bracketing it with these two keys
endOfPromptKey = 249  #This key is put in code after the prompt is put there

#The text made by ChatGPT 3.5
textG35 = "Сонячний ліс влітку був найкращим місцем для відпочинку. Під високими кронами дерев розквітали барвисті квіти, а гілки м'яко похитувалися на вітрі. Коли я вирушив на свою прогулянку, сонце щедро розпростерло свої промені, створюючи мерехтливі плями на стежці. Початкові кроки привели мене до гаю з величезними дубами і кленами. Легкий шум листя надавав лісу власний ритм, а спів пташок створював мелодійну симфонію. Змінюючи напрямок руху, я опинився біля маленького струмка. Вода пливла спокійно і чисто, а на його березі росли жовті лілії, що розцвітали в воді. Пройшовши глибше в ліс, я натрапив на лісовий майданчик, де сонячні промені змішувалися з тінями дерев. Тут я вирішив влаштувати пікнік, розклавши ковдру і розмістивши корзину з їжею. Затишна атмосфера і шум природи зробили мій обід надзвичайно приємним. Найбільше мене вразила зустріч з дикою природою. Барвисті метелики летіли навколо мене, а білки стежили за моїми рухами. Поруч із струмком, я випив прохолодної води і відчував себе щасливим. Сонце спускалося за горизонт, і коли я повертався додому, усе ще чув спів пташок у лісі. Та прогулянка лісом влітку залишила в мене незабутні враження і нагадала, наскільки важливо дбати про нашу природу і насолоджуватися її красою."
wordsG35 = ClearElements(textG35.split())  #Split the text by spaces and clear it from delimiters and any symbols except for words themselves
promptG35 = wordsG35[:promptSizeUkr]  #The beginning of the text to keep safe for decoding afterwards
#The key - starts as a prompt, but transforms after each word encoded
keyG35 = wordsG35[:promptSizeUkr]
#The encoded text - prompt is put here, and all other codes are too
codeG35 = wordsG35[:promptSizeUkr]
codeG35.append(endOfPromptKey)
currentIndexG35 = promptSizeUkr + 1

textG2 = generatedText
print(textG2)
wordsG2 = ClearElements(textG2.split())  #The same inits for the text generated by ChatGPT 2
promptG2 = wordsG2[:promptSizeUkr]
keyG2 = wordsG2[:promptSizeUkr]
codeG2 = wordsG2[:promptSizeUkr]
codeG2.append(endOfPromptKey)
currentIndexG2 = promptSizeUkr + 1
#Human-written text
#Link to the text: https://glazastik.com/%d0%bb%d1%96%d1%81-%d0%b2%d0%bb%d1%96%d1%82%d0%ba%d1%83-%d1%82%d0%b2%d1%96%d1%80-%d0%be%d0%bf%d0%b8%d1%81-%d1%82%d0%b5%d0%ba%d1%81%d1%82-%d1%80%d0%be%d0%b7%d0%bf%d0%be%d0%b2%d1%96%d0%b4%d1%8c/
textH = "Ліс прекрасний у будь-який час року, але особливо влітку. Коли заходиш у нього, то відразу ж змінюється настрій, стає легше дихати. У спекотну погоду відчувається приємна прохолода. В холодний день, навпаки, нагріті стовбури дерев віддають своє тепло. При вході відразу ж уся увага звертається на дерева. У змішаному лісі можна побачити величні дуби, стрункі сосни та пірамідальні ялини. Дуже гарно виглядають білокорі берези, вони завжди радісно шелестять листячком на вітрі. Під деревами навкруги ростуть чагарники. Всі рослини тягнуться вгору, ближче до світла й сонця. Ліс влітку дуже багатий різноманітною рослинністю. Всюди висока трава. Особливо приємно знайти сонячну галявину, яка вся в квітах конюшини, ромашки, гвоздики, кашок. Багато квітучих лікарських рослин: звіробій, материнка, деревій. А можна й наштовхнутися на таке місце, де навколо червоніють ягоди суниці. Потрібно тільки не лінуватися, щоб її збирати. У густих заростях зустрічається чорниця. Опівдні температура повітря стає високою, квітучі трави починають пахнути. Дуже приємно лягти серед цих квітів та вдихати аромати квітів. А ще — дивитися, як по блакитному небу пропливають хмари та вгадувати, на що вони схожі. Отже, ліс влітку — осередок краси та єднання з природою. Обов’язково відвідайте це зелене царство!"

wordsH = ClearElements(textH.split())  #The same inits for the human-written text
promptH = wordsH[:promptSizeUkr]
keyH = wordsH[:promptSizeUkr]
codeH = wordsH[:promptSizeUkr]
codeH.append(endOfPromptKey)
currentIndexH = promptSizeUkr + 1

print(promptG35)
print(promptG2)
print(promptH)
print(len(wordsG35))
print(len(textG2.split()))
print(len(textH.split()))
print(len(textG35))
print(len(textG2))
print(len(textH))

Сонячний літній ліс, в якому я люблю гуляти, – це місце, куди хочеться повертатися знову і знову. Я ніколи не забуду цього чарівного місця, де я народилася і виросла. Тут я знайшла себе, тут я зустріла свою другу половинку. Я завжди мріяла про те, щоб у мене була своя сім’я. Але коли я стала дорослою, я зрозуміла, що це не так просто. І мені захотілося зробити щось більше, ніж просто бути мамою. У мене з’явилося багато друзів, з якими я можу поділитися своїми захопленнями, своїми думками і переживаннями. Мені дуже подобається подорожувати, і я дуже рада, коли мої друзі приїжджають до мене в гості. Коли я приїжджаю до них, мені хочеться поділитися з ними своєю любов’ю і турботою про мене. Тому я завжди буду рада знайомству з новими людьми, які захочуть поділитися своїм теплом і ніжністю з моїми близькими людьми. Це дуже важливо для мене, тому що я відчуваю, як мої близькі люди відчувають мою любов і турботу. А це дуже приємно для нас обох. Адже ми любимо один одного і дуже хочемо бути р

In [42]:
#Before encoding-compression-decoding, analyze the texts
import bz2

print("ENTROPY")
#The higher Entropy means the text is less likely to be artificial
print("ChatGPT 3.5: " + str(ShannonEntropy(textG35)))  #Generated text (ChatGPT 3.5)
print("ChatGPT 2: " + str(ShannonEntropy(textG2)))  #Generated text (ChatGPT 2)
print("Human: " + str(ShannonEntropy(textH)))  #Seemingly human text

print("AVERAGE")
#As the English language model works with Ukrainian sentences, it is used here
procGenerText = engnlp(textG35)
procG2Text = engnlp(textG2)
procHumanText = engnlp(textH)
sumG = 0
for phrase in procGenerText._.phrases:
    sumG = sumG + phrase.rank
sumG2 = 0
for phrase in procG2Text._.phrases:
    sumG2 = sumG2 + phrase.rank
sumH = 0
for phrase in procHumanText._.phrases:
    sumH = sumH + phrase.rank
#Phrase rank Average. The higher Average means the text is less likely to be artificial
averageG = sumG / len(procGenerText._.phrases)
print("ChatGPT 3.5: " + str(averageG))
averageG2 = sumG2 / len(procG2Text._.phrases)
print("ChatGPT 2: " + str(averageG2))
averageH = sumH / len(procHumanText._.phrases)
print("Human: " + str(averageH))

print("VARIANCE")
listG = []
for phrase in procGenerText._.phrases:
    listG.append(phrase.rank)
listG2 = []
for phrase in procG2Text._.phrases:
    listG2.append(phrase.rank)
listH = []
for phrase in procHumanText._.phrases:
    listH.append(phrase.rank)

#The lower Variance means the text is less likely to be edited
print("ChatGPT 3.5: " + str(statistics.variance(listG)))
print("ChatGPT 2: " + str(statistics.variance(listG2)))
print("Human: " + str(statistics.variance(listH)))

print("COMPRESSION RATE (ORIGINAL)")
#The better the text is compressed, the more artificial it may be
compressedG = bz2.compress(textG35.encode())
compressedH = bz2.compress(textH.encode())
decompressedG = bz2.decompress(compressedG).decode()
decompressedH = bz2.decompress(compressedH).decode()
compressedG2 = bz2.compress(textG2.encode())
decompressedG2 = bz2.decompress(compressedG2).decode()
print("ChatGPT 3.5: " + str(len(textG35)/len(compressedG)))
print("ChatGPT 2: " + str(len(textG2)/len(compressedG2)))
print("Human: " + str(len(textH)/len(compressedH)))

ENTROPY
ChatGPT 3.5: 4.595582247453061
ChatGPT 2: 4.614975892208419
Human: 4.695354496057342
AVERAGE
ChatGPT 3.5: 0.05771758587525143
ChatGPT 2: 0.06091264171128362
Human: 0.04519870368921074
VARIANCE
ChatGPT 3.5: 0.0004942431435823742
ChatGPT 2: 0.000819278385091761
Human: 0.0004650122788464069
COMPRESSION RATE (ORIGINAL)
ChatGPT 3.5: 1.6108949416342413
ChatGPT 2: 1.706786171574904
Human: 1.5736906211936663


In [ ]:
#IF NEEDED, it works
#ENCODE the prompt
#In the code, only the first word will be written, other ones will be coded

def PromptEncode(code, key, numberOfWords, prompt, model, tokenizer, maxIndex):
  counter = 0
  for word in prompt:
    if counter == 0:  #The first word must be put into code
      key.append(word)
      code.append(word)
      counter = counter + 1
    else:
      #Combine the words already present in key to make the string
      string = CombineWordsInString(key)
      #Encode the string
      encoded_text = tokenizer(string, return_tensors="pt")
      #Get the possible next words
      wordsWithProbs = FindWordsDictionary(encoded_text, maxIndex * 2, model, tokenizer)
      #Append the encoded word to the code
      CodeAppending(prompt, wordsWithProbs, counter, maxIndex, code)
      key.append(word)  #And put the original word into the key
      counter = counter + 1
  code.append(endOfPromptKey)

In [ ]:
#IF NEEDED, it works
PromptEncode(codeG35, keyG35, promptSizeUkr, promptG35, mUkr, tUkr, maxIndexUkr)
print(promptG35)
print(keyG35)
print(codeG35)

['І', 'ось', 'ми', 'тут', 'робимо', 'лабораторну', 'роботу']
['І', 'ось', 'ми', 'тут', 'робимо', 'лабораторну', 'роботу']
['І', 254, 'ось', 255, 3, 23, 95, 254, 'лабораторну', 255, 0]


In [46]:
#ENCODE first
def Encode(currentIndex, words, key, tokenizer, model, maxIndex, code):
  while currentIndex < len(words):
    #The key is kept in separate words. For encoding, they must be put into one string
    stringUkr = CombineWordsInString(key)
    #Encode the string
    encoded_text = tokenizer(stringUkr, return_tensors="pt")
    #Get words with the highest probabilities to appear after the string
    wordsWithProbsUkr = FindWordsDictionary(encoded_text, maxIndexUkr + 1, model, tokenizer)
    #Append the code of the word (or the word itself) to the code
    CodeAppending(words, wordsWithProbsUkr, currentIndex, maxIndex, code)
    #Remove the first element from the key
    key.pop(0)
    #Add the new word to the key
    key.append(words[currentIndex])
    #Increase the currentIndex
    currentIndex = currentIndex + 1

In [47]:
Encode(currentIndexG35, wordsG35, keyG35, tUkr, mUkr, maxIndexUkr, codeG35)
Encode(currentIndexG2, wordsG2, keyG2, tUkr, mUkr, maxIndexUkr, codeG2)
Encode(currentIndexH, wordsH, keyH, tUkr, mUkr, maxIndexUkr, codeH)

print(codeG35)
print(codeG2)
print(codeH)

['Сонячний', 'ліс', 'влітку', 'був', 'найкращим', 'місцем', 'для', 'відпочинку', 'Під', 'високими', 'кронами', 'дерев', 'розквітали', 'барвисті', 'квіти', 249, 254, 'гілки', 255, 254, "м'яко", 255, 254, 'похитувалися', 255, 3, 254, 'вітрі', 255, 254, 'Коли', 255, 7, 254, 'вирушив', 255, 2, 6, 42, 254, 'сонце', 255, 254, 'щедро', 255, 254, 'розпростерло', 255, 0, 0, 254, 'створюючи', 255, 254, 'мерехтливі', 255, 6, 0, 254, 'стежці', 255, 254, 'Початкові', 255, 9, 25, 2, 0, 254, 'гаю', 255, 4, 254, 'величезними', 255, 254, 'дубами', 255, 2, 254, 'кленами', 255, 254, 'Легкий', 255, 69, 30, 107, 26, 254, 'власний', 255, 34, 63, 39, 254, 'пташок', 255, 254, 'створював', 255, 254, 'мелодійну', 255, 254, 'симфонію', 255, 254, 'Змінюючи', 255, 33, 2, 40, 254, 'опинився', 255, 3, 37, 254, 'струмка', 255, 20, 254, 'пливла', 255, 139, 1, 100, 87, 7, 4, 2, 49, 48, 254, 'лілії', 255, 7, 254, 'розцвітали', 255, 3, 254, 'воді', 255, 254, 'Пройшовши', 255, 235, 3, 4, 2, 254, 'натрапив', 255, 0, 254, '

In [19]:
#IF NEEDED, it works
#The prompt must also be DECODED
#This variable is the amount of words / codes to skip from the beginning when the text decoding will start
decodeIndexUkr = 0

def PromptDecode(encodedWords, decodedText, decodedKey, model, tokenizer, maxIndex, promptLength, input):
  global decodeIndexUkr
  counter = 0
  while 1:
    word = encodedWords[decodeIndexUkr]
    #The first word is taken directly
    if decodeIndexUkr == 0:
      decodedText.append(word)
      decodedKey.append(word)
      counter = counter + 1
      decodeIndexUkr = decodeIndexUkr + 1
    else:
      #If the end of prompt is reached, break the loop
      if word == endOfPromptKey:
        decodeIndexUkr = decodeIndexUkr + 1
        break
      #If a bracket is detected, check if, without the first letter, the word between the brackets is a number
      if word == unknownKeyS or word == notFitKeyS or word == numberWordKeyS:
        tempWord = encodedWords[decodeIndexUkr + 1].replace(encodedWords[decodeIndexUkr + 1][0], "", 1)
        if tempWord.isdigit():  #If it is, put the number into the decrypted text
          decodedText.append(tempWord)
          decodedKey.append(tempWord)
        else:  #If it is not, put the word into the decrypted text without changes
          decodedText.append(encodedWords[decodeIndexUkr + 1])
          decodedKey.append(encodedWords[decodeIndexUkr + 1])
        decodeIndexUkr = decodeIndexUkr + 3  #The index is increased by 3 - two brackets and the word
        counter = counter + 1  #Word counter is increased by 1
      #Otherwise, decode the word
      else:
        #Get the key as one string
        stringKey = CombineWordsInString(decodedKey)
        #Encode it
        encoded_stringUkr = tokenizer(stringKey, return_tensors="pt")
        #Get possible next words
        possiblewordsG35 = FindWordsDictionary(encoded_stringUkr, maxIndex * 2, model, tokenizer)
        indexUkr = input[decodeIndexUkr]
        word = ""
        #Find the word on a position set by the code
        for i, key in enumerate(possiblewordsG35.keys()):
          if i == indexUkr:
            word = key
            break
        #Append the word to the text and the key
        decodedText.append(word)
        decodeKey.append(word)
        decodeIndexUkr = decodeIndexUkr + 1
        counter = counter + 1

In [44]:
def StringsToIntsInACode(codeParts):
  counter = 0
  for codePart in codeParts:
    if codePart.isdigit():
      codeParts[counter] = int(codePart)
    counter = counter + 1
  return codeParts

In [45]:
#Now COMPRESS the code and DECOMPRESS it

stringToCompressG35 = CombineWordsInString(codeG35)
compressedG35 = bz2.compress(stringToCompressG35.encode())
print(len(compressedG35))
stringToCompressG2 = CombineWordsInString(codeG2)
compressedG2 = bz2.compress(stringToCompressG2.encode())
print(len(compressedG2))
stringToCompressH = CombineWordsInString(codeH)
compressedH = bz2.compress(stringToCompressH.encode())
print(len(compressedH))

decompressedG35 = bz2.decompress(compressedG35).decode()
print(len(decompressedG35))
codeMadeG35 = decompressedG35.split()
print(codeMadeG35)
decompressedG2 = bz2.decompress(compressedG2).decode()
print(len(decompressedG2))
codeMadeG2 = decompressedG2.split()
print(codeMadeG2)
decompressedH = bz2.decompress(compressedH).decode()
print(len(decompressedH))
codeMadeH = decompressedH.split()
print(codeMadeH)

codeMadeG35 = StringsToIntsInACode(codeMadeG35)
codeMadeG2 = StringsToIntsInACode(codeMadeG2)
codeMadeH = StringsToIntsInACode(codeMadeH)
print(codeMadeG35)
print(codeMadeG2)
print(codeMadeH)

print("COMPRESSION RATE (ENCODED)")
print("ChatGPT 3.5: " + str(len(stringToCompressG35)/len(compressedG)))
print("ChatGPT 2: " + str(len(stringToCompressG2)/len(compressedG2)))
print("Human: " + str(len(stringToCompressH)/len(compressedH)))

137
133
125
112
['Сонячний', 'ліс', 'влітку', 'був', 'найкращим', 'місцем', 'для', 'відпочинку', 'Під', 'високими', 'кронами', 'дерев', 'розквітали', 'барвисті', 'квіти', '249']
89
['Сонячний', 'літній', 'ліс', 'в', 'якому', 'я', 'люблю', 'гуляти', '–', 'це', 'місце', 'куди', 'хочеться', 'повертатися', 'знову', '249']
91
['Ліс', 'прекрасний', 'у', 'будь-який', 'час', 'року', 'але', 'особливо', 'влітку', 'Коли', 'заходиш', 'у', 'нього', 'то', 'відразу', '249']
['Сонячний', 'ліс', 'влітку', 'був', 'найкращим', 'місцем', 'для', 'відпочинку', 'Під', 'високими', 'кронами', 'дерев', 'розквітали', 'барвисті', 'квіти', 249]
['Сонячний', 'літній', 'ліс', 'в', 'якому', 'я', 'люблю', 'гуляти', '–', 'це', 'місце', 'куди', 'хочеться', 'повертатися', 'знову', 249]
['Ліс', 'прекрасний', 'у', 'будь-який', 'час', 'року', 'але', 'особливо', 'влітку', 'Коли', 'заходиш', 'у', 'нього', 'то', 'відразу', 249]
COMPRESSION RATE (ENCODED)
ChatGPT 3.5: 0.14526588845654995
ChatGPT 2: 0.6691729323308271
Human: 0.7

In [21]:
inputG35 = codeG35[:]
textG35 = []
decodekeyG35 = []
inputG2 = codeG2[:]
textG2 = []
decodekeyG2 = []
inputH = codeH[:]
textH = []
decodekeyH = []

#PromptDecode(inputUkr, textG35, decodekeyG35, mUkr, tUkr, maxIndexUkr, promptSizeUkr)
decodeIndexG35 = 0
print(textG35)
print(decodekeyG35)
print(decodeIndexG35)
decodeIndexG2 = 0
print(textG2)
print(decodekeyG2)
print(decodeIndexG2)
decodeIndexH = 0
print(textH)
print(decodekeyH)
print(decodeIndexH)

[]
[]
0
[]
[]
0
[]
[]
0


In [22]:
#DECODE the text
#The decoding goes through the same process as encoding
def Decode(decodeIndex, code, decodeKey, text, input):
  endOfPromptReached = False
  while decodeIndex < len(code):
    #If the word is the code of the end of prompt, don't put it in code, but change the endOfPromptReached variable
    if input[decodeIndex] == endOfPromptKey:
      decodeIndex = decodeIndex + 1
      endOfPromptReached = True
    #If the end of prompt is not reached, put the word unchanged
    elif not endOfPromptReached:
      text.append(code[decodeIndex])
      decodeKey.append(code[decodeIndex])
      decodeIndex = decodeIndex + 1
    #If the code is one of the brackets, put the next word unchanged...
    elif code[decodeIndex] == unknownKeyS or code[decodeIndex] == notFitKeyS or code[decodeIndex] == numberWordKeyS:
      tempWordUkr = code[decodeIndex + 1].replace(code[decodeIndex + 1][0], "", 1)
      #...Except if it's the number
      if tempWordUkr.isdigit() or re.match(r"[-+]?(?:\d*\.*\d+)", tempWordUkr):
        text.append(tempWordUkr)
        decodeKey.append(tempWordUkr)
      else:
        text.append(code[decodeIndex + 1])
        decodeKey.append(code[decodeIndex + 1])
      #First element is removed from the key, as the new one is placed into it
      decodeKey.pop(0)
      decodeIndex = decodeIndex + 3
    #If the code is not the bracket, it must be decoded
    else:
      #Make the string from the key
      stringkey = CombineWordsInString(decodeKey)
      #Encode
      encoded_stringUkr = tUkr(stringkey, return_tensors="pt")
      #Get possible words
      possiblewords = FindWordsDictionary(encoded_stringUkr, maxIndexUkr + 1, mUkr, tUkr)
      indexUkr = input[decodeIndex]
      word = ""
      #Find index of the word needed
      for i, key in enumerate(possiblewords.keys()):
        if i == indexUkr:
          word = key
          break
      #Append it to the key. Remove the first element from the key
      text.append(word)
      decodeKey.append(word)
      decodeKey.pop(0)
      decodeIndex = decodeIndex + 1

In [23]:
Decode(decodeIndexG35, codeG35, decodekeyG35, textG35, inputG35)
print(textG35)
Decode(decodeIndexG2, codeG2, decodekeyG2, textG2, inputG2)
print(textG2)
Decode(decodeIndexH, codeH, decodekeyH, textH, inputH)
print(textH)

['Сонячний', 'ліс', 'влітку', 'був', 'найкращим', 'місцем', 'для', 'відпочинку', 'Під', 'високими', 'кронами', 'дерев', 'розквітали', 'барвисті', 'квіти', 'гілки', "м'яко", 'похитувалися', 'на', 'вітрі', 'Коли', 'я', 'вирушив', 'на', 'свою', 'прогулянку', 'сонце', 'щедро', 'розпростерло', 'свої', 'промені', 'створюючи', 'мерехтливі', 'плями', 'на', 'стежці', 'Початкові', 'кроки', 'привели', 'мене', 'до', 'гаю', 'з', 'величезними', 'дубами', 'і', 'кленами', 'Легкий', 'шум', 'листя', 'надавав', 'лісу', 'власний', 'ритм', 'а', 'спів', 'пташок', 'створював', 'мелодійну', 'симфонію', 'Змінюючи', 'напрямок', 'руху', 'я', 'опинився', 'біля', 'маленького', 'струмка', 'Вода', 'пливла', 'спокійно', 'і', 'чисто', 'а', 'на', 'його', 'березі', 'росли', 'жовті', 'лілії', 'що', 'розцвітали', 'в', 'воді', 'Пройшовши', 'глибше', 'в', 'ліс', 'я', 'натрапив', 'на', 'лісовий', 'майданчик', 'де', 'сонячні', 'промені', 'змішувалися', 'з', 'тінями', 'дерев', 'Тут', 'я', 'вирішив', 'влаштувати', 'пікнік', 'ро

In [ ]:
for i in range(0, dataframe.shape[0] - 1):
  #Get the text and init all the needed arrays
  print("Text " + str(i))
  text = dataframe[1].iloc[i]
  words = ClearElements(text.split())
  prompt = words[:promptSizeUkr]
  key = words[:promptSizeUkr]
  code = words[:promptSizeUkr]
  code.append(endOfPromptKey)
  currentIndex = promptSizeUkr + 1
  print("Number of words: " + str(len(words)))
  print("Number of characters: " + str(len(text)))
  #Analyze the text
  print("Entropy: " + str(ShannonEntropy(text)))
  procText = engnlp(text)
  sum = 0
  for phrase in procText._.phrases:
    sum = sum + phrase.rank
  average = sum / len(procText._.phrases)
  print("Average: " + str(average))
  listT = []
  for phrase in procText._.phrases:
    listT.append(phrase.rank)
  print("Variance: " + str(statistics.variance(listT)))
  compressed = bz2.compress(text.encode())
  print("Compression Rate (Original): " + str(len(text)/len(compressed)))
  #Encode the text
  Encode(currentIndex, words, key, tUkr, mUkr, maxIndexUkr, code)
  #Compress the text
  stringToCompress = CombineWordsInString(code)
  compressed = bz2.compress(stringToCompress.encode())
  print("Number of characters (Encoded): " + str(len(stringToCompress)))
  print("Number of characters (Encoded, Compressed): " + str(len(compressed)))
  print("Compression Rate (Encoded): " + str(len(stringToCompress)/len(compressed)))
  print("")

Text 0
Number of words: 143
Number of characters: 834
Entropy: 4.642262413593914
Average: 0.06948368037759553
Variance: 0.0005617057589052754
Compression Rate (Original): 1.5676691729323309
Number of characters (Encoded): 699
Number of characters (Encoded, Compressed): 359
Compression Rate (Encoded): 1.947075208913649

Text 1
Number of words: 226
Number of characters: 1272
Entropy: 4.650908711830497
Average: 0.060753819940388046
Variance: 0.0008402505063645537
Compression Rate (Original): 1.6649214659685865
Number of characters (Encoded): 1003
Number of characters (Encoded, Compressed): 485
Compression Rate (Encoded): 2.0680412371134023

Text 2
Number of words: 182
Number of characters: 962
Entropy: 4.656544258581663
Average: 0.06378857234543575
Variance: 0.0007633565069896946
Compression Rate (Original): 1.5441412520064206
Number of characters (Encoded): 756
Number of characters (Encoded, Compressed): 405
Compression Rate (Encoded): 1.8666666666666667

Text 3
Number of words: 163
Numb

In [ ]:
#I need to split the text into sentences AND IN UKRAINIAN
#Library link: https://github.com/lang-uk/tokenize-uk/tree/master

#!pip install tokenize_uk

import tokenize_uk
from tokenize_uk import tokenize_sents

text = "Перше речення. Друге речення. Речення №1.5. Речення ім. мене."
sentences = tokenize_sents(text)
for sent in sentences:
  print(sent)

Перше речення.
Друге речення.
Речення №1.5.
Речення ім. мене.


In [ ]:
#What I also want to try is the Dynamic Key - encoding using SENTENCES and not WORDS

#The text we're to encode
text = "І ось ми тут, робимо лабораторну роботу 1 українською мовою. На нас чекають ще інші справи, але ми тут і робимо це. А на додачу ще й ключі робимо з речень. Щоб життя зовсім легким не здавалося."
sentenceArray = tokenize_sents(text)  #All sentences in the text
keyNumber = 2  #How many sentences are to be used as a key
#WE ASSUME the sentence array is LARGER than the prompt we need
promptSentenceArray = sentenceArray[:keyNumber]  #The beginning of the text we're to keep safe
sentencesKey = sentenceArray[:keyNumber]  #The sentences to act as the key - starts as a prompt, but transforms after each sentence encoded

maxIndex = 200  #Max code we can insert in the code list

#NEEDED encode parts of the prompt as well! Or is this a MAYBE?
code = promptSentenceArray[:]  #Well, code itself - prompt is put here, and all other codes are too
unknownKeyS = 253
unknownKeyE = 254  #If the word doesn't appear in the tensor - we put it in the code, bracketing it with these two keys
notFitKeyS = 251
notFitKeyE = 252  #If the word appeared in tensor, but didn't fit within the code limit - we put it in code, bracketing with these two keys
numberWordKeyS = 249
numberWordKeyE = 250  #If the word is a number and doesn't have to be decrypted while decoding - we bracket it with these two keys
endOfSentenceKey = 255  #When we work with sentences, we need to keep the delimiter

print(sentenceArray)
print(promptSentenceArray)
print(sentencesKey)
print(code)

['І ось ми тут, робимо лабораторну роботу 1 українською мовою.', 'На нас чекають ще інші справи, але ми тут і робимо це.', 'А на додачу ще й ключі робимо з речень.', 'Щоб життя зовсім легким не здавалося.']
['І ось ми тут, робимо лабораторну роботу 1 українською мовою.', 'На нас чекають ще інші справи, але ми тут і робимо це.']
['І ось ми тут, робимо лабораторну роботу 1 українською мовою.', 'На нас чекають ще інші справи, але ми тут і робимо це.']
['І ось ми тут, робимо лабораторну роботу 1 українською мовою.', 'На нас чекають ще інші справи, але ми тут і робимо це.']


In [ ]:
#This one works WAAAAY too long. Guess I'll drop it for now
currentIndex = keyNumber
while currentIndex < len(sentenceArray):
  stringKey = CombineSentencesToString(sentencesKey)
  inSentenceIndex = 0
  stringToEncode = stringKey
  while inSentenceIndex < len(sentenceArray[currentIndex]):
    words = ClearElements(sentenceArray[currentIndex].split())
    stringToEncode = stringToEncode + " " + words[inSentenceIndex]
    encoded_string = tUkr(stringToEncode, return_tensors="pt")
    wordsWithProbs = FindWordsDictionary(encoded_string, maxIndex * 2, mUkr, tUkr)

    CodeAppending(words, wordsWithProbs, inSentenceIndex, maxIndex, code)
  print(sentencesKey)
  sentencesKey.pop(0)
  sentencesKey.append(sentenceArray[currentIndex])
  #Increase the currentIndex
  currentIndex = currentIndex + 1
  print(sentencesKey)
  print(code)

KeyboardInterrupt: ignored

In [ ]:
#And now we need to compress the encoded thing
import zlib

def StringsToIntsInACode(codeParts):
  counter = 0
  for codePart in codeParts:
    if codePart.isdigit():
      codeParts[counter] = int(codePart)
    counter = counter + 1
  return codeParts

#Darn, to compress, I need to put the code in a string
stringToCompressUkr = CombineWordsInString(codeG35)
print(stringToCompressUkr)
compressed = zlib.compress(stringToCompressUkr.encode())
print(compressed)
print(len(compressed))
decompressed = zlib.decompress(compressed).decode()
print(decompressed)
print(len(decompressed))
codeMade = decompressed.split()
print(codeMade)
#Not much of a compression...
#Maybe I need a more powerful library?

print(StringsToIntsInACode(codeMade))

І ось ми тут робимо лабораторну роботу 250 n1 251 254 українською 255 0 208 254 нас 255 1 50 57 46 123 1 68 1 58 4
b"x\x9c5\x8c\xc1\r\xc3 \x10\x04[\xd9\x12\x80\x80MU\xa9!\x06\xc9\xaf(\xdf\xb4a\xff,'\xa6\x86\xbd\x8e\xb2 \xe5qZ\xed\xdc\xdcq\x05\x9b-\xf6\x04\xbf<`\xc5\xaa\x15\xd8\x83\x8d;\x0f\xb1\x06~\xb8\xa94\xc1\xcd\xca\xc8\xcb\xea\xdfi\xfd\x04!9\xdc\xbd\xa2O\x84U\x9eC\x7fK\xd5s\x9e\xf2^Z%8\x04\x97\x87\xc4K\xc22\xa0\x87\xee\xd3\x8c8\xc1\x87\x9b\xea\x94;\xcb\x88?(\xe2N9"
120
І ось ми тут робимо лабораторну роботу 250 n1 251 254 українською 255 0 208 254 нас 255 1 50 57 46 123 1 68 1 58 4
114
['І', 'ось', 'ми', 'тут', 'робимо', 'лабораторну', 'роботу', '250', 'n1', '251', '254', 'українською', '255', '0', '208', '254', 'нас', '255', '1', '50', '57', '46', '123', '1', '68', '1', '58', '4']
['І', 'ось', 'ми', 'тут', 'робимо', 'лабораторну', 'роботу', 250, 'n1', 251, 254, 'українською', 255, 0, 208, 254, 'нас', 255, 1, 50, 57, 46, 123, 1, 68, 1, 58, 4]


In [ ]:
#What I want to try is detect not only Integers, but also Floats and keep them unchanged
import re

def KeepFloats(inputParts):
  outputParts = []
  for part in inputParts:
    if re.match(r"[-+]?(?:\d*\.*\d+)", part):
      outputParts.append("n" + part)
    else:
      outputParts.append(part)
  return outputParts

floatString = "One of my friends is 24.5 years old and another one is 20 years old"
floatStringParts = floatString.split()
returnedParts = KeepFloats(floatStringParts)
print(returnedParts)

clearedParts = ClearElements(floatStringParts)
print(clearedParts)

['One', 'of', 'my', 'friends', 'is', 'n24.5', 'years', 'old', 'and', 'another', 'one', 'is', 'n20', 'years', 'old']
['One', 'of', 'my', 'friends', 'is', '24.5', 'years', 'old', 'and', 'another', 'one', 'is', '20', 'years', 'old']


In [ ]:
#What I need is getting the probabilities of words occuring in the text

def LetterProbabilities(inputString):
  words = collections.Counter(inputString)
  amount = words.total()
  probs = {}
  counter = 0
  for word in words:
    probs.update({word : words[word] / amount* 100})
  return probs

wordProbs = LetterProbabilities(text)
for word in wordProbs:
  print(word + ' | ' + str(wordProbs[word]))

L | 0.4405286343612335
e | 11.45374449339207
t | 6.607929515418502
’ | 0.4405286343612335
s | 4.845814977973569
  | 17.180616740088105
a | 7.929515418502203
y | 3.524229074889868
w | 3.0837004405286343
h | 3.9647577092511015
v | 0.881057268722467
l | 4.845814977973569
n | 4.405286343612335
g | 0.881057268722467
u | 3.9647577092511015
m | 0.881057268722467
o | 4.405286343612335
d | 2.643171806167401
b | 1.762114537444934
r | 3.0837004405286343
i | 2.643171806167401
c | 2.643171806167401
f | 1.762114537444934
5 | 0.4405286343612335
, | 2.2026431718061676
. | 0.881057268722467
N | 0.4405286343612335
p | 1.3215859030837005
x | 0.4405286343612335


In [ ]:
#Okay, I get a list of letters. Now I'll try to get the WORDS
#Need to remove the spaces as well
import re

delimiters = [',', '|', ';', '.', ':', ' ']

def WordProbabilities(inputString):
  pattern = r'[,|;.: ]'
  allWords = re.split(pattern, inputString)
  words = collections.Counter(allWords)
  for word in words:
    if word in delimiters:
      del words[word]
  amount = words.total()
  probs = {}
  for word in words:
      probs.update({word : words[word] / amount * 100})
  return probs

wordProbs = WordProbabilities(text)
for word in wordProbs:
  print(word + ' | ' + str(wordProbs[word]))

Let’s | 2.127659574468085
say | 2.127659574468085
we | 4.25531914893617
have | 2.127659574468085
a | 4.25531914893617
language | 2.127659574468085
model | 4.25531914893617
that | 2.127659574468085
has | 2.127659574468085
been | 2.127659574468085
trained | 2.127659574468085
with | 2.127659574468085
vocabulary | 2.127659574468085
of | 4.25531914893617
only | 2.127659574468085
5 | 2.127659574468085
words | 2.127659574468085
sunny | 2.127659574468085
 | 14.893617021276595
day | 2.127659574468085
beautiful | 4.25531914893617
scenery | 4.25531914893617
clouds | 2.127659574468085
Now | 2.127659574468085
want | 2.127659574468085
to | 2.127659574468085
calculate | 2.127659574468085
the | 6.382978723404255
perplexity | 2.127659574468085
when | 2.127659574468085
it | 2.127659574468085
sees | 2.127659574468085
phrase | 2.127659574468085


In [ ]:
#I'll try a bit different approach
#YAAAY ^-^
#Maybe also make the IgnoreCase? DONE
import math

delimiters = [',', '|', ';', '.', ':', ' ']

def CountElements(elements):
  dictionary = {}
  for element in elements:
    element = element.lower()
    if element[-1] in delimiters:
        element = element[:-1]
    if element in dictionary:
        dictionary[element] += 1
    else:
        dictionary.update({element: 1})
  return dictionary

def WordProbabilities2(inputString):
  words = CountElements(inputString.split())
  counter = 0
  amount = math.fsum(words.values())
  probs = {}
  for word in words:
      probs.update({word : words[word] / amount * 100})
  return probs

wordProbs = WordProbabilities2(text)
sortedWordProbs = dict(sorted(wordProbs.items(), key=lambda x:x[1]))
for word in sortedWordProbs:
  print(word + ' | ' + str(sortedWordProbs[word]))

let’s | 2.5
say | 2.5
have | 2.5
language | 2.5
that | 2.5
has | 2.5
been | 2.5
trained | 2.5
with | 2.5
vocabulary | 2.5
only | 2.5
5 | 2.5
words | 2.5
sunny | 2.5
day | 2.5
clouds | 2.5
now | 2.5
want | 2.5
to | 2.5
calculate | 2.5
perplexity | 2.5
when | 2.5
it | 2.5
sees | 2.5
phrase | 2.5
we | 5.0
a | 5.0
model | 5.0
of | 5.0
beautiful | 5.0
scenery | 5.0
the | 7.5


In [ ]:
#Maybe what I'll use is Prediction by Partial Matching
#But I'll start with Huffman coding. I've used it once
#Link where I've taken the code from: https://favtutor.com/blogs/huffman-coding

from collections import Counter

class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right
    def children(self):
        return self.left, self.right
    def __str__(self):
        return self.left, self.right


def huffman_code_tree(node, binString=''):
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, binString + '0'))
    d.update(huffman_code_tree(r, binString + '1'))
    return d


def make_tree(nodes):
    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
    return nodes[0][0]

text = 'BCAADDDCCACACAC'
freq = dict(Counter(text))
freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
node = make_tree(freq)
encoding = huffman_code_tree(node)
for i in encoding:
    print(f'{i} : {encoding[i]}')

C : 0
B : 100
D : 101
A : 11


In [ ]:
#One more text to test out

text = "Let’s say we have a language model that has been trained with a vocabulary of only 5 words sunny, day, beautiful, scenery, clouds. Now, we want to calculate the perplexity of the model when it sees the phrase beautiful scenery."
freq = dict(Counter(text))
freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
node = make_tree(freq)
encoding = huffman_code_tree(node)
for i in encoding:
    print(f'{i} : {encoding[i]}')

o : 0000
n : 0001
, : 00100
c : 00101
l : 0011
s : 0100
i : 01010
d : 01011
e : 011
p : 100000
m : 1000010
g : 1000011
r : 10001
w : 10010
x : 10011000
N : 10011001
. : 1001101
f : 100111
t : 1010
y : 10110
b : 101110
v : 1011110
L : 10111110
5 : 101111110
’ : 101111111
u : 11000
h : 11001
a : 1101
  : 111


In [ ]:
#Pretty nice
#Guess I need to enter the deep water now. The PPM algorithm

#I feel like there's a library I can use...
#It's called arithmetic_compressor
#Link: https://pypi.org/project/arithmetic-compressor/
#Or there is another one, pyppmd, Link: https://pyppmd.readthedocs.io/en/latest/

#Guess I need a sort-of different prediction model. I need to predict WORDS, not letters, as in the regular model

!pip install arithmetic_compressor
from arithmetic_compressor import AECompressor
from arithmetic_compressor.models import StaticModel

letterProbs = LetterProbabilities(text)
model = StaticModel(letterProbs)

# create an arithmetic coder
coder = AECompressor(model)

# encode some data
N = len(text)
compressed = coder.compress(text)

# print the compressed data
print(compressed)
print(len(compressed))

#now decode
decoded = coder.decompress(compressed, N)
print(decoded)
print(len(decoded))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 

In [ ]:
#Allright, we have the encoded letters. I doubt I can encode words using my text, I need a larger one
#BUT I can test the library on letters first
from arithmetic_compressor.models import PPMModel, MultiPPM

letters = Counter(text)
model = PPMModel(letters, k = 5) # no need to pass in probabilities, only symbols

# create an arithmetic coder
coder = AECompressor(model)

# encode some data
data = text[0: len(text) // 2]
compressed = coder.compress(data)

# print the compressed data
print(compressed)
print(len(compressed))

#now decode
decoded = coder.decompress(compressed, len(text)//2)
print(decoded)
print(len(decoded))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 

In [ ]:
#The previous model gives an error if it doesn't know the symbol it gets. The context there is strict. I need a more flexible version
#BUUUUT if we get a long text, there will be all letters
#But not all words, though, this is troubling

#I need to try to predict the next word. There are some methods online to try out
#I'll use this link as a code source: https://www.ris-ai.com/predict-next-word-with-python
#Link to the dataset: https://www.kaggle.com/datasets/rowemorehouse/googleplaystoreuserreviews

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')
dataset = pd.read_csv('/content/gdrive/MyDrive/Kaggle/googleplaystore_user_reviews.csv')
dataset.head()

Mounted at /content/gdrive


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [ ]:
datasetPart = pd.Series(dataset['Translated_Review'], dtype=str)
datasetPart.head()

0    I like eat delicious food. That's I'm cooking ...
1      This help eating healthy exercise regular basis
2                                                  nan
3           Works great especially going grocery store
4                                         Best idea us
Name: Translated_Review, dtype: object

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = []
for string in datasetPart:
  words.extend(tokenizer.tokenize(string))
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['I', 'like', 'eat', 'delicious', 'food']
That


In [ ]:
#ERROR out of RAM
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1
print(X[0][0])

In [ ]:
#What's the problem with finding the word?
wordUkr = 'мені'
dictionary = {tUkr.decode(idx): prob for idx, prob in zip(topk_next_tokensUkr.indices, topk_next_tokensUkr.values)}
clearedKeys = []
for item in dictionary.keys():
  clearedKeys.append(ClearString(item))
dictionary = dict(zip(clearedKeys, dictionary.values()))
if wordUkr in dictionary.keys():
  print("Got the key")
else:
  print("Not the key")

Got the key


In [ ]:
#I'll try to make the code

promptSize = 7  #How long do we want our prompt to be - how many words are to be inserted
maxIndex = 250  #Max code we can insert in the code list

#The text we're to encode
text = "This used to be ok. Now there are way too many characters but nothing to do."
#Second part of the text: Too many attractions but nowhere to put them. Developers have fallen way behind opening an additional land.
words = ClearElements(text.split())
prompt = words[:promptSize]  #The beginning of the text we'll keep safe for decoding
key = words[:promptSize]  #The key - starts as a prompt, but transforms after each word encoded
#NEEDED encode parts of the prompt as well! Or is this a MAYBE?
code = words[:promptSize]  #Well, code itself - prompt is put here, and all other codes are too
unknownKeyS = 254
unknownKeyE = 255  #If the word doesn't appear in the tensor - we put it in the code, bracketing it with these two keys
notFitKeyS = 252
notFitKeyE = 253  #If the word appeared in tensor, but didn't fit within the 250-code limit - we put it in code, bracketing with these two keys
numberWordKeyS = 250
numberWordKeyE = 251  #If the word is a number and doesn't have to be decrypted while decoding

currentIndex = promptSize
print(prompt)

['This', 'used', 'to', 'be', 'ok', 'Now', 'there']


In [ ]:
t = GPT2TokenizerFast.from_pretrained("gpt2")
m = GPT2LMHeadModel.from_pretrained("gpt2")

encoded_text = t("I enjoy walking in the", return_tensors="pt")

#1. step to get the logits of the next token
with torch.inference_mode():
  outputs = m(**encoded_text)

next_token_logits = outputs.logits[0, -1, :]
print(next_token_logits.shape)

# 2. step to convert the logits to probabilities
next_token_probs = torch.softmax(next_token_logits, -1)

# 3. step to get the top 10
topk_next_tokens= torch.topk(next_token_probs, 20)

#putting it together
print(*[(t.decode(idx), prob) for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)], sep="\n")

torch.Size([50257])
(' park', tensor(0.1590))
(' woods', tensor(0.1003))
(' streets', tensor(0.0418))
(' dark', tensor(0.0312))
(' door', tensor(0.0296))
(' street', tensor(0.0239))
(' rain', tensor(0.0217))
(' city', tensor(0.0189))
(' same', tensor(0.0150))
(' halls', tensor(0.0135))
(' field', tensor(0.0128))
(' middle', tensor(0.0124))
(' garden', tensor(0.0106))
(' neighborhood', tensor(0.0103))
(' snow', tensor(0.0095))
(' forest', tensor(0.0092))
(' parks', tensor(0.0090))
(' open', tensor(0.0085))
(' world', tensor(0.0076))
(' hallway', tensor(0.0069))


In [ ]:
while currentIndex < len(words):
  string = CombineWordsInString(key)
  encoded_text = t(string, return_tensors="pt")

  wordsWithProbs = FindWordsDictionary(encoded_text, maxIndex + 1, m, t)
  CodeAppending(words, wordsWithProbs, currentIndex, maxIndex, code)
  print(key)
  #Now we got the word and added it to the code. We need to...
  #Remove the first element from the key
  key.pop(0)
  #Add the word to the key
  key.append(words[currentIndex])
  #Increase the currentIndex
  currentIndex = currentIndex + 1
  #And repeat the process again
  print(key)
  print(code)

TypeError: ignored

In [ ]:
#This was the ENCODING part
#Now I need the DECODING part as well. This is where the prompt will be used

input = code[:]
text = prompt[:]
decodeKey = prompt[:]
decodeIndex = promptSize
print(input)
print(text)

while decodeIndex < len(code):
  if code[decodeIndex] == unknownKeyS or code[decodeIndex] == notFitKeyS or code[decodeIndex] == numberWordKeyS:
    text.append(code[decodeIndex + 1])
    decodeKey.append(code[decodeIndex + 1])
    decodeKey.pop(0)
    decodeIndex = decodeIndex + 3
  else:
    stringKey = CombineWordsInString(decodeKey)
    encoded_string = t(stringKey, return_tensors="pt")
    possibleWords = FindWordsDictionary(encoded_string, maxIndex + 1, m, t)
    index = input[decodeIndex]
    text.append(list(possibleWords)[index])
    decodeKey.append(list(possibleWords)[index])
    decodeKey.pop(0)
    decodeIndex = decodeIndex + 1

print(text)

['This', 'used', 'to', 'be', 'ok', 'Now', 'there', 1, 90, 0, 0, 42, 56, 47, 3, 2]
['This', 'used', 'to', 'be', 'ok', 'Now', 'there']
['This', 'used', 'to', 'be', 'ok', 'Now', 'there', 'are', 'way', 'too', 'many', 'characters', 'but', 'nothing', 'to', 'do']
